# Configuration

The `logging` package contains many facilities to set up the configuration for it. There is even a separate module for it, `logging.config`. This page is focused on such options.

Check [documentation of the `logging.config` module](https://docs.python.org/3/library/logging.config.html#module-logging.config).

In [4]:
import logging

## dictionary

By using `logging.config.dictConfig` function you can cofigure your logging system using python dictionaries. As a consequence, any other format chasing the `key:value` format can be used (`yaml`, `json`, `xml` and so on).

It's the configuration method recommended [by official documentation](https://docs.python.org/3/howto/logging.html#configuring-logging) for new applications and deployments.

**Note** it may seem that converting yaml, json, etc. into dictionaries and then using `dictConfig` is a crutch, but this is the method recommended by the official documentation.

### yaml to dict

It's pretty trivial task all you need is:

- [Basic understanding of the yaml files](https://www.cloudbees.com/blog/yaml-tutorial-everything-you-need-get-started);
- [pyaml package](https://pypi.org/project/PyYAML/) - to get yaml file as python dictionary.

The following cell defines a yaml file that can be converted to a dictionary with logging configuration. It'll be a logger that prints to the error stream with the pattern `<-- %(message)s -->`.

In [52]:
%%writefile configuration_files/example.yaml
version: 1
formatters:
    simpleFormatter :
        format : "<-- %(message)s -->"
handlers:
    consoleHandler:
        class: "logging.StreamHandler"
        formatter: "simpleFormatter"
loggers:
    simpleExample:
        level: DEBUG
        handlers:
            - "consoleHandler"

Overwriting configuration_files/example.yaml


The next cell loads the previous yaml as a dictionary and shows that all is well.

In [54]:
import yaml
with open("configuration_files/example.yaml", "r") as f:
    conf_dict = yaml.safe_load(f)


logging.config.dictConfig(conf_dict)
logger = logging.getLogger("simpleExample")
logger.warning("wow it finally works")

<-- wow it finally works -->


## `ini` file

You can store your configuration in the `.ini` files.

Read more about `.ini` files:

- [configparser](https://docs.python.org/3/library/configparser.html#module-configparser) python package;
- [yaml vs ini](https://www.honeybadger.io/blog/python-ini-vs-yaml/) page that describes structure of the ini and yaml files.

Here is example of suck `.ini` file.

In [5]:
%%writefile configuration_files/example_logging.ini
[loggers]
keys=root,simpleExample

[handlers]
keys=consoleHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_simpleExample]
level=DEBUG
handlers=consoleHandler
qualname=simpleExample
propagate=0

[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=simpleFormatter
args=(sys.stdout,)

[formatter_simpleFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s

Overwriting configuration_files/example_logging.ini


You can use the `logging.config.fileConfig` function to load the configuration file - all it's entities will be added to your program. The following cell uses the config from the previous cell to define it's loggers.

In [7]:
logging.config.fileConfig('configuration_files/example_logging.ini')
logger = logging.getLogger('simpleExample')
logger.debug("test")

2024-05-07 15:34:09,986 - simpleExample - DEBUG - test
